# Task B: Stack Overflow Developer Survey 2025 Analytics

You are provided with the latest developer survey results from Stack Overflow. Your task is to perform analytics on the survey to extract insights about the programming industry.

## Setup
If you are in google colab, you should just be able to run the cell below. Otherwise find the conda `environment.yml` file provided with all the dependencies.

In [1]:
%pip install pandas
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.4 MB/s eta 0:00:00
  Using cached numpy-2.3.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 11.9 MB/s eta 0:00:0000:0100:01
Using cached numpy-2.3.3-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Reading the data

Find a utility class below to read the data for you.

In [ ]:
import csv
from typing import List, Dict, Any, Optional
from pathlib import Path

RESPONSE_ID_FIELD_NAME = "ResponseId"
QUESTION_ID_FIELD_NAME = "qid"

class SurveyDataReader:
    """
    Read and process Stack Overflow Developer Survey data.
    """

    def __init__(self, schema_file: str, data_file: str):
        self.schema = self._parse_schema(schema_file)
        self.data = self._parse_data(data_file)

    def _parse_schema(self, schema_file: str) -> List[Dict[str, str]]:
        schema = []
        schema_path = Path(schema_file).resolve()
        with open(schema_path, mode="r", encoding="utf-8-sig") as file:
            reader = csv.DictReader(file)
            schema = [row for row in reader]
        return schema

    def _parse_data(self, data_file: str) -> List[Dict[str, Any]]:
        data = []
        data_path = Path(data_file).resolve()
        with open(data_path, mode="r") as file:
            reader = csv.DictReader(file)
            data = [row for row in reader]
        return data

    def get_schema(self) -> List[Dict[str, str]]:
        return self.schema

    def get_data(self) -> List[Dict[str, Any]]:
        return self.data

    def get_question_by_id(self, qid: str) -> Optional[Dict[str, str]]:
        for question in self.schema:
            if question[QUESTION_ID_FIELD_NAME] == qid:
                return question
        return None

    def get_responses_for_question(self, qname: str) -> List[Any]:
        return [response[qname] for response in self.data if qname in response]

    def get_response_by_id(self, response_id: str | int) -> Optional[Dict[str, Any]]:
        response_id_str = str(response_id)
        for response in self.data:
            if response[RESPONSE_ID_FIELD_NAME] == response_id_str:
                return response
        return None

## Getting to know the data reader

In [18]:
SURVEY_SUBDIR = "stack-overflow-developer-survey-2025"
SCHEMA_RELATIVE_PATH = f"{SURVEY_SUBDIR}/survey_results_schema.csv"
DATA_RELATIVE_PATH = f"{SURVEY_SUBDIR}/survey_results_public_cleaned.csv"

reader = SurveyDataReader(SCHEMA_RELATIVE_PATH, DATA_RELATIVE_PATH)

In [20]:
print(reader.get_schema())

print(len(reader.get_data()))

print(reader.get_data()[0]) # Be careful when trying to output the data, there's lots of it!

[{'qid': 'QID18', 'qname': 'TechEndorse_1', 'question': 'What attracts you to a technology or causes you to endorse it (most to least important)?', 'type': 'RO', 'sub': 'AI integration or AI Agent capabilities', 'sq_id': '1'}, {'qid': 'QID18', 'qname': 'TechEndorse_2', 'question': 'What attracts you to a technology or causes you to endorse it (most to least important)?', 'type': 'RO', 'sub': 'Easy-to-use API', 'sq_id': '2'}, {'qid': 'QID18', 'qname': 'TechEndorse_3', 'question': 'What attracts you to a technology or causes you to endorse it (most to least important)?', 'type': 'RO', 'sub': 'Robust and complete API', 'sq_id': '3'}, {'qid': 'QID18', 'qname': 'TechEndorse_4', 'question': 'What attracts you to a technology or causes you to endorse it (most to least important)?', 'type': 'RO', 'sub': 'Customizable and manageable codebase', 'sq_id': '4'}, {'qid': 'QID18', 'qname': 'TechEndorse_5', 'question': 'What attracts you to a technology or causes you to endorse it (most to least impor

## Questions

1. Print all of the questions asked in the developer survey

In [17]:
unique_qids = {row["qid"] for row in reader.get_schema()}
for qid in unique_qids:
    print(reader.get_question_by_id(qid)['question'])

Did you spend time in the last year learning AI programming or AI-enabled tooling on your own or at work?
Where do you live?
Was the community platform you want to use not listed above?  List below, separated by a comma and a space for each, what you used in the past year.
How many years of professional work experience do you have? Please round to the nearest whole number, excluding any decimal points.  If your answer is '0', please leave blank.
Which <strong>development environments and AI-enabled code editing tools</strong> did you use regularly over the past year, and which do you want to work with over the next year? Please check all that apply.
Was the programming language you use not listed above?  List below, separated by a comma and a space for each, the programming language(s) <strong>you used in the past year</strong>.
Was the programming language you want to use not listed above? List below, separated by a comma and a space for each, the programming language(s) <strong>you w

2. Which age range has the most responses in the survey?

In [55]:
df = pd.DataFrame(reader.get_data())

most_common_age = df['Age'].value_counts().idxmax()

print(f"Most response age range: {most_common_age}")

Most response age range: 25-34 years old


3. How many survey respondents do we know definitely work for a company larger than Marshall Wace? (Feel free to ask one of us if you don't know how large Marshall Wace is!)

In [58]:
#Marshall Wace has 797 employees
largeCompanys = ['1,000 to 4,999 employees', '5,000 to 9,999 employees', '10,000 or more employees']

total = df['OrgSize'].isin(largeCompanys).sum()
print(total)

10715


4. How many survey respondents had less than 1 year of coding experience before (or outside of) coding for their profession?

In [ ]:
# Assume NA in YearsCode means less than a year of coding experience
# less than 1 year of coding experience outside of coding for profession = same number in YearsCode and WorkExp

df['YearsCode'] = pd.to_numeric(df['YearsCode'], errors='coerce')
df['WorkExp'] = pd.to_numeric(df['WorkExp'], errors='coerce')

total = (df['YearsCode'].isna() | 
         ((~df['WorkExp'].isna()) & ((df['YearsCode'] - df['WorkExp']) == 0))).sum()

print(total)

11390


5. Of the people who had 1 or more years of coding experience outside of coding professionally, what is the average number of years they spent coding outside of work? For simplicity, you can consider only the people who have given an exact number of years they have spent coding in both columns (i.e. excluding those with over 50 or less than 1 year)

In [69]:
mask = df['YearsCode'].notna() & df['WorkExp'].notna() & (df['YearsCode'] > df['WorkExp'])
avg_years = (df.loc[mask, 'YearsCode'] - df.loc[mask, 'WorkExp']).mean()

print(f"Average year of coding outside of work is {avg_years:.4f}")

Average year of coding outside of work is 5.9135


6. What is the median annual total compensation of those who specified their compensation in USD

In [77]:
df['CompTotal'] = pd.to_numeric(df['CompTotal'], errors='coerce')

mask = df['Currency'] == 'USD United States dollar'
median_tc = df.loc[mask, 'CompTotal'].median()
print(f"Median total compensation in usd is ${median_tc:,.0f}")


Median total compensation in usd is $130,000


7. Which programming language has respondents with the highest annual compensation in USD? If a response lists multiple languages, you can attribute the compensation to each language in the response.

In [80]:
usd_df = df[df['Currency'] == 'USD United States dollar'].copy()

lang_df = (usd_df.assign(Language=usd_df['LanguageHaveWorkedWith'].str.split(';')).explode('Language'))

median_comp = (lang_df.groupby('Language')['CompTotal'].median())

top_lang = median_comp.idxmax()
top_salary = median_comp.max()

print(f"The highest median total compensation in USD grouped by language is {top_lang}: ${top_salary:,.0f}")

The highest median total compensation in USD grouped by language is Scala: $180,000


## Bonus Task: SurveyDataReader

`SurveyDataReader` is a basic class that allows you to access the underlying survey data programmatically. The class is implemented with basic data structures and no external dependencies hence there is plenty of room for optimisation. Try to improve the speed of basic operations and add some of your own by potentially leveraging a package such as [NumPy](https://numpy.org/).